# Shakespear Transformer

In [1]:
import requests

# url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
vocab_url = "http://qwone.com/~jason/20Newsgroups/vocabulary.txt"
text_url = "http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz"
r = requests.get(vocab_url)
text = r.text

##### Char to Integer Encoding

In [2]:
chars = sorted(list(set(text)))
data_size, vocab_size = len(text), len(chars)

In [3]:
stoi = {ch:i for i,ch in enumerate(chars)} # string to index
itos = {i:ch for i,ch in enumerate(chars)} # index to string
encode_int = lambda x: [stoi[c] for c in x] # encode text to array of ints
decode_int = lambda x: ''.join([itos[c] for c in x]) # decode array of ints to text

##### BPE Tokenisation

In [4]:
import tiktoken

encoder = tiktoken.get_encoding("gpt2")
assert encoder.decode(encoder.encode("Hello World")) == "Hello World"

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

data = torch.tensor(encoder.encode(text), dtype=torch.long)
dataset = torch.utils.data.TensorDataset(data)

train_size = int(0.75 * len(dataset))
test_size = int(0.15 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_set, test_set, val_set = random_split(dataset, [train_size, test_size, val_size])

train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)
val_loader = DataLoader(val_set, batch_size=1, shuffle=False)

context_window = 8

In [6]:
x = train_set[:context_window]
y = train_set[1:context_window+1]
for t in range(context_window):
    context = x[:t+1]
    target = y[t]
    print(f"With input {context} the target is {target}")

With input (tensor([   66,   568,   198,   261,   620, 36902,   297, 24903]),) the target is tensor([  568,   198,   261,   620, 36902,   297, 24903,  7230])


IndexError: tuple index out of range